# Vibrational structure

In [ ]:
from qiskit_nature.settings import settings

settings.dict_aux_operators = True

## Introduction 

The molecular Hamiltonian is 

$$
\mathcal{H} = - \sum_I \frac{\nabla_{R_I}^2}{M_I} - \sum_i \frac{\nabla_{r_i}^2}{m_e} - \sum_I\sum_i  \frac{Z_I e^2}{|R_I-r_i|} + \sum_i \sum_{j>i} \frac{e^2}{|r_i-r_j|} + \sum_I\sum_{J>I} \frac{Z_I Z_J e^2}{|R_I-R_J|}
$$

Because the nuclei are much heavier than the electrons they do not move on the same time scale and therefore, the behavior of nuclei and electrons can be decoupled. This is the Born-Oppenheimer approximation.

Within the Born-Oppenheimer approximation, a molecular wave function is factorized as a product of an electronic part, which is the solution of the electronic Schroedinger equation, and a vibro-rotational one, which is the solution of the nuclear Schroedinger equation in the potential energy surface (PES) generated by sampling the eigenvalues of the electronic Schroedinger equation for different geometries.

The nuclear Schroedinger equation is usually solved in two steps, in analogy with its electronic counterpart. 
A single-particle basis (the basis functions are called, in this case, modals) is obtained either by the harmonic approximation applied to the PES or from a vibrational self-consistent field (VSCF) calculation. 
Vibrational anharmonic correlations are added a-posteriori with perturbative or variational approaches.
The latter include Vibrational Configuration Interaction (VCI) and Vibrational Coupled Cluster (VCC) for highly-accurate anharmonic energies. 
The main advantage of VCI and VCC over alternative approaches (such as perturbation theories) is that their accuracy can be systematically improved towards the complete basis set limit for a given PES. 
However, their applicability is limited to small molecules with up to about 10 atoms due to their unfavorable scaling with system size.

To tackle the scaling problem we would like to use quantum algorithms.

The nuclear Schroedinger equation is
$$
\mathcal{H}_{\text{vib}} |\Psi_{n}\rangle = E_{n} |\Psi_{n}\rangle
$$

The so-called Watson Hamiltonian (neglecting vibro-rotational coupling terms) is
$$
  \mathcal{H}_\text{vib}(Q_1, \ldots, Q_L) 
    = - \frac{1}{2} \sum_{l=1}^{L} \frac{\partial^2}{\partial Q_l^2} + V(Q_1, \ldots, Q_L)
$$
where $Q_l$ are the harmonic mass-weighted normal coordinates.

$\mathcal{H}_\text{vib}$ must be mapped to an operator that acts on the states of a given set of $N_q$ qubits in order to calculate its eigenfunctions on quantum hardware.
In electronic structure calculations, the mapping is achieved by expressing the non-relativistic electronic Hamiltonian in second quantization, \textit{i.e.} by projecting it onto the complete set of antisymmetrized occupation number vectors (ONV) generated by a given (finite) set of orbitals.
To encode the vibrational Hamiltonian in an analogous second quantization operators, we expand the potential $V(Q_1, \ldots, Q_L)$ with the $n$-body expansion as follows:

$$
  V(Q_1, \ldots, Q_L) = V_0 + \sum_{l=1}^L V^{[l]}(Q_l) 
    + \sum_{l<m}^L V^{[l,m]}(Q_l, Q_m) + \sum_{l<m<n}^L V^{[l,m,n]}(Q_l, Q_m, Q_n) + \ldots
$$

where $V_0$ is the electronic energy of the reference geometry, the one-mode term $V^{[l]}(Q_l)$ represents the variation of the PES upon change of the $l$-th normal coordinate from the equilibrium position.
Similarly, the two-body potential $V^{[l,m]}(Q_l, Q_m)$ represents the change in the exact PES upon a simultaneous displacement along the $l$-th and $m$-th coordinates. 
Often, including terms up to three-body in the $L$-body expansion is sufficient to obtain an accuracy of about 1~cm$^{-1}$. We highlight that the many-body expansion of the potential operator defining the Watson Hamiltonian contains arbitrarily high coupling terms. This is a crucial difference compared to the non-relativistic electronic-structure Hamiltonian that contains only pairwise interactions.

A flexible second quantization form of the Watson Hamiltonian is obtained within the so-called n-mode representation. Let us assume that each mode $l$ is described by a $N_l$-dimensional basis set $S_l$ defined as follows:

$$
  \mathcal{S}_l = \{ \phi_1^{(l)} (Q_l) , \ldots , \phi_{N_l}^{(l)} (Q_l) \} \, .
$$

The $n$-mode wave function can be expanded in the product basis $\mathcal{S} = \otimes_{i=1}^L \mathcal{S}_i$ as the following CI-like expansion:

$$
  |\Psi\rangle = \sum_{k_1=1}^{N_1} \cdots \sum_{k_L=1}^{N_L} C_{k_1,\ldots,k_L} 
    \phi_{k_1}^{(1)}(Q_1) \cdots \phi_{k_L}^{(L)}(Q_L) \, ,
$$

The many-body basis function $\phi_{k_1}^{(1)}(Q_1) \cdots \phi_{k_L}^{(L)}(Q_L)$ are encoded within the so-called $n$-mode second quantization as occupation-number vectors (ONVs) as follows:

$$
  \phi_{k_1}(Q_1) \cdots \phi_{k_L}(Q_L)
                      \equiv  |0_1 \cdots 1_{k_1} \cdots 0_{N_1},
                                   0_1 \cdots 1_{k_2} \cdots 0_{N_2}, 
                                   \cdots , 
                                   0_1 \cdots 1_{k_L} \cdots 0_{N_L}\rangle \, .
$$

The ONV defined above is, therefore, the product of $L$ mode-specific ONVs, each one describing an individual mode. Since each mode is described by one and only one basis function, the occupation of each mode-specific ONV is one.
From a theoretical perspective, each mode can be interpreted as a distinguishable quasi-particle (defined as phonons in solid-state physics). Distinguishability arises from the fact that the PES is not invariant by permutation of two modes, also in this case unlike the Coulomb interaction between two equal particles. From this perspective, a molecule can be interpreted as a collection of $L$ indistinguishable particles that interact through the PES operator.

Based on this second-quantization representation we introduce a pair of creation and annihilation operators per mode $l$ \textit{and} per basis function $k_l$ defined as:

$$
  \begin{aligned}
    a_{k_l}^\dagger |\cdots, 0_1 \cdots 0_{k_l} \cdots 0_{N_l}, \cdots\rangle 
      &=  | \cdots, 0_1 \cdots 1_{k_l} \cdots 0_{N_l}, \cdots\rangle \\
    a_{k_l}^\dagger | \cdots, 0_1 \cdots 1_{k_l} \cdots 0_{N_l}, \cdots\rangle &=  0 \\
    a_{k_l} | \cdots, 0_1 \cdots 1_{k_l} \cdots 0_{N_l}, \cdots\rangle
     &= | \cdots, 0_1 \cdots 0_{k_l} \cdots 0_{N_l}, \cdots\rangle \\
    a_{k_l} | \cdots, 0_1 \cdots 0_{k_l} \cdots 0_{N_l}, \cdots\rangle &=  0 \\
  \end{aligned}
$$

with

$$
  \begin{aligned}
    \left[ a_{k_l}^\dagger, a_{h_m}^\dagger \right] &= 0 \\
    \left[ a_{k_l}, a_{h_m} \right] &= 0 \\
    \left[ a_{k_l}^\dagger, a_{h_m} \right] &= \delta_{l,m} \, , \delta_{k_l,h_m}
  \end{aligned}
$$

The second quantization form is obtained by expressing the potential as 

$$
 \begin{aligned}
  \mathcal{H}_\text{vib}^{SQ} =& \sum_{l=1}^L 
    \sum_{k_l,h_l}^{N_l} \langle \phi_{k_l} | T(Q_l) + V^{[l]}(Q_l) | \phi_{h_l} \rangle a_{k_l}^+ a_{h_l} \\
 +& \sum_{l<m}^L \sum_{k_l,h_l}^{N_l} \sum_{k_m,h_m}^{N_m}
    \langle \phi_{k_l} \phi_{k_m} | V^{[l,m]}(Q_l, Q_m) | \phi_{h_l} \phi_{h_m} \rangle 
    a_{k_l}^+ a_{k_m}^+ a_{h_l} a_{h_m} + \cdots
 \end{aligned}
$$

We highlight here the difference between the operators defined here above and the electronic structure one. First, as we already mentioned, the potential contains (in principle) three- and higher-body coupling terms that lead to strings with six (or more) second-quantization operators. 
Moreover, the Hamiltonian conserves the number of particles for each mode, as can be seen from the fact that the number of creation and annihilation operators for a given mode is the same in each term. Nevertheless, different modes are coupled by two- (and higher) body terms containing SQ operators belonging to different modes $l$ and $m$.

Reference: Ollitrault, Pauline J., et al., arXiv:2003.12578 (2020).

Compute the electronic potential

Solving the ESE for different nuclear configurations to obtain the PES function $V(Q_1, \ldots, Q_L)$. So far Qiskit gives the possibility to approximate the PES with a quartic force field. 
$$
V(Q_1, \ldots, Q_L) = \frac{1}{2}  \sum_{ij} k_{ij} Q_i Q_j
                  + \frac{1}{6}  \sum_{ijk} k_{ijk} Q_i Q_j Q_k
                  + \frac{1}{16} \sum_{ijkl} k_{ijkl} Q_i Q_j Q_k Q_l
$$
The advantage of such form for the PES is that the anharmonic force fields ($k_{ij}$, $k_{ijk}$, $k_{ijkl}$) can be calculated by finite-difference approaches. For methods for which the nuclear energy Hessian can be calculated analytically with response theory-based methods (such as HF and DFT), the quartic force field can be calculated by semi-numerical differentiation as:
$$
k_{ijk} = \frac{H_{ij}(+\delta Q_k) - H_{ij}(-\delta Q_k)}{2\delta Q_k}
$$
and
$$
k_{ijkl} = \frac{H_{ij}(+\delta Q_k+\delta Q_l) - H_{ij}(+\delta Q_k-\delta Q_l)
                    -H_{ij}(-\delta Q_k+\delta Q_l) + H_{ij}(-\delta Q_k+\delta Q_l)}
                    {4\delta Q_k \delta Q_l}
$$
Such numerical procedure is implemented, for instance, in the Gaussian suite of programs.

In practice this can be done with Qiskit using the GaussianForceDriver. 
       

In [1]:
from qiskit_nature.drivers.second_quantization import GaussianForcesDriver

# if you ran Gaussian elsewhere and already have the output file
driver = GaussianForcesDriver(logfile="aux_files/CO2_freq_B3LYP_631g.log")

# if you want to run the Gaussian job from Qiskit
# driver = GaussianForcesDriver(
#                 ['#p B3LYP/6-31g Freq=(Anharm) Int=Ultrafine SCF=VeryTight',
#                  '',
#                  'CO2 geometry optimization B3LYP/6-31g',
#                  '',
#                  '0 1',
#                  'C  -0.848629  2.067624  0.160992',
#                  'O   0.098816  2.655801 -0.159738',
#                  'O  -1.796073  1.479446  0.481721',
#                  '',
#                  ''

## Map to a qubit Hamiltonian

Now that we have an approximation for the potential, we need to write the Hamiltonian in second quantization. To this end we need to select a modal basis to calculate the one-body integrals $\langle\phi_{k_i}| V(Q_i) | \phi_{h_i} \rangle$, two-body integrals $\langle\phi_{k_i} \phi_{k_j}| V(Q_i,Q_j) | \phi_{h_i} \phi_{h_j} \rangle$... 

In the simplest case, the $\phi$ functions are the harmonic-oscillator eigenfunctions for each mode. The main advantage of this choice is that the integrals of a PES expressed as a Taylor expansion are easy to calculate with such basis. A routine for computing these integrals is implemented in Qiskit. 

The bosonic operator, $\mathcal{H}_\text{vib}^{SQ}$, is then created and must be mapped to a qubit operator. The direct mapping introduced in the first section of this tutorial can be used is Qiskit as follows:

In [2]:
from qiskit_nature.problems.second_quantization import VibrationalStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import DirectMapper

vibrational_problem = VibrationalStructureProblem(driver, num_modals=2, truncation_order=2)
second_q_ops = vibrational_problem.second_q_ops()

The Vibrational operator for the problem now reads as

In [3]:
print(second_q_ops["VibrationalEnergy"])

  NIIIIIII * (1268.0676746875001+0j)
+ INIIIIII * (3813.8767834375008+0j)
+ IINIIIII * (705.8633818750001+0j)
+ II+-IIII * (-46.025705898886045+0j)
+ II-+IIII * (-46.025705898886045+0j)
+ IIINIIII * (2120.1145593750007+0j)
+ IIIINIII * (238.31540750000005+0j)
+ IIIIINII * (728.9613775000003+0j)
+ IIIIIINI * (238.31540750000005+0j)
+ IIIIIIIN * (728.9613775000003+0j)
+ NINIIIII * (4.942542500000002+0j)
+ NI+-IIII * (-88.20174216876333+0j)
+ NI-+IIII * (-88.20174216876333+0j)
+ NIINIIII * (14.827627500000007+0j)
+ INNIIIII * (14.827627500000007+0j)
+ IN+-IIII * (-264.60522650629+0j)
+ IN-+IIII * (-264.60522650629+0j)
+ ININIIII * (44.482882500000024+0j)
+ NIIINIII * (-10.205891250000004+0j)
+ INIINIII * (-30.617673750000016+0j)
+ IININIII * (-4.194299375000002+0j)
+ II+-NIII * (42.67527310283147+0j)
+ II-+NIII * (42.67527310283147+0j)
+ IIINNIII * (-12.582898125000007+0j)
+ NIIIINII * (-30.61767375000002+0j)
+ INIIINII * (-91.85302125000007+0j)
+ IINIINII * (-12.582898125000007+0j)
+ II+

In the previous cell we defined a bosonic transformation to express the Hamiltonian in the harmonic modal basis, with 2 modals per mode with the potential truncated at order 2 and the 'direct' boson to qubit mapping. 
The calculation is then ran as:

In [4]:
qubit_converter = QubitConverter(mapper=DirectMapper())
qubit_op = qubit_converter.convert(second_q_ops["VibrationalEnergy"])

print(qubit_op)

4854.200029687493 * IIIIIIII
- 342.57516687500015 * ZIIIIIII
- 111.85586312500007 * IZIIIIII
- 342.57516687500015 * IIZIIIII
+ 15.97128750000001 * ZIZIIIII
+ 5.323762500000003 * IZZIIIII
- 111.85586312500007 * IIIZIIII
+ 5.323762500000003 * ZIIZIIII
+ 1.7745875000000009 * IZIZIIII
- 1049.719110937499 * IIIIZIII
- 9.437173593750005 * ZIIIZIII
- 3.1457245312500017 * IZIIZIII
- 9.437173593750005 * IIZIZIII
- 3.1457245312500017 * IIIZZIII
- 349.4856346875002 * IIIIIZII
- 3.1457245312500017 * ZIIIIZII
- 1.0485748437500004 * IZIIIZII
- 3.1457245312500017 * IIZIIZII
- 1.0485748437500004 * IIIZIZII
- 1860.5306717187502 * IIIIIIZI
- 22.963255312500017 * ZIIIIIZI
- 7.654418437500004 * IZIIIIZI
- 22.963255312500017 * IIZIIIZI
- 7.654418437500004 * IIIZIIZI
+ 11.120720625000006 * IIIIZIZI
+ 3.706906875000002 * IIIIIZZI
- 618.5645973437502 * IIIIIIIZ
- 7.654418437500005 * ZIIIIIIZ
- 2.551472812500001 * IZIIIIIZ
- 7.654418437500005 * IIZIIIIZ
- 2.551472812500001 * IIIZIIIZ
+ 3.706906875000002 * IIII

To have a different number of modals per mode:

In [5]:
vibrational_problem = VibrationalStructureProblem(driver, num_modals=3, truncation_order=2)
second_q_ops = vibrational_problem.second_q_ops()

qubit_converter = QubitConverter(mapper=DirectMapper())

qubit_op = qubit_converter.convert(second_q_ops["VibrationalEnergy"])

print(qubit_op)

10788.719982656237 * IIIIIIIIIIII
- 541.6731217187498 * ZIIIIIIIIIII
- 315.1932645312502 * IZIIIIIIIIII
- 102.72856234375004 * IIZIIIIIIIII
- 541.6731217187498 * IIIZIIIIIIII
+ 44.36468750000001 * ZIIZIIIIIIII
+ 26.618812500000015 * IZIZIIIIIIII
+ 8.872937500000004 * IIZZIIIIIIII
- 13.32410345498359 * XIXIIIIIIIII
- 13.32410345498359 * YIYIIIIIIIII
- 12.54822855058883 * XIXZIIIIIIII
- 12.54822855058883 * YIYZIIIIIIII
- 315.1932645312502 * IIIIZIIIIIII
+ 26.61881250000001 * ZIIIZIIIIIII
+ 15.97128750000001 * IZIIZIIIIIII
+ 5.323762500000003 * IIZIZIIIIIII
- 7.528937130353299 * XIXIZIIIIIII
- 7.528937130353299 * YIYIZIIIIIII
- 102.72856234375004 * IIIIIZIIIIII
+ 8.872937500000003 * ZIIIIZIIIIII
+ 5.323762500000003 * IZIIIZIIIIII
+ 1.7745875000000009 * IIZIIZIIIIII
- 2.509645710117766 * XIXIIZIIIIII
- 2.509645710117766 * YIYIIZIIIIII
- 1730.9391493749995 * IIIIIIZIIIII
- 26.21437109375001 * ZIIIIIZIIIII
- 15.728622656250007 * IZIIIIZIIIII
- 5.242874218750002 * IIZIIIZIIIII
- 26.2143710937

Now that the Hamiltonian is ready, it can be used in a quantum algorithm to find information about the vibrational structure of the corresponding molecule. Check out our tutorials on Ground State Calculation and Excited States Calculation to learn more about how to do that in Qiskit Nature!

In [6]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright

/home/oss/Files/Qiskit/src/qiskit-terra/main/qiskit/tools/jupyter/__init__.py:133: RuntimeWarning: matplotlib can't be found, ensure you have matplotlib and other visualization dependencies installed. You can run '!pip install qiskit-terra[visualization]' to install it from jupyter
  warnings.warn(
